In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf


from datetime import date, timedelta, datetime # Date Functions
import time
import os

import google.cloud.aiplatform as aip
from google.cloud import bigquery
from google.oauth2 import service_account

from google.cloud import aiplatform

print(tf.__version__)
print(aip.__version__)

2023-11-10 09:48:49.851642: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarni

2.12.0
1.26.0


# Get explanations locally in Vertex AI Workbench user-managed notebooks
* https://cloud.google.com/vertex-ai/docs/predictions/overview
* https://cloud.google.com/vertex-ai/docs/explainable-ai/configuring-explanations-feature-based
* https://cloud.google.com/vertex-ai/docs/explainable-ai/getting-explanations#local-explanations
* https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-online-predictions#aiplatform_explain_tabular_sample-python_vertex_ai_sdk
* https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-batch-predictions
* https://cloud.google.com/vertex-ai/docs/explainable-ai/tensorflow

# Use SDK  VertextAi Sample Code
* https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_online_explain_get_metadata.ipynb
* https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_online_explain.ipynb
* https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_batch_explain.ipynb



# Varaible

In [2]:
#label_multi_severity
project_id="pongthorn"
region='asia-southeast1'

model_dir='gs://demo2-tf-incident-pongthorn/demo_model_tf' # demo
# model_with_meta_dir='gs://demo2-tf-incident-pongthorn/demo_model_meta_tf'



# Load model from GCS

In [3]:
#model_with_meta_dir='model_with_meta'
local_model= tf.keras.models.load_model(model_dir)
print(local_model.tensorflow_version)

2.12.0


# Prediction 

In [4]:
sample= {"sla": ["24x7 6Hrs Resolution Time"], "product_type": ["Storage"], "brand": ["NetApp"], 
         "service_type": ["Incident"], "incident_type": ["General Incident"], 
         "open_to_close_hour": [1268.9333333333334], "response_to_resolved_hour": [1268.8]}

print(sample)
              
print("===============================================================================================================")    
print("convert pain data to serdor as input to predict")    
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
print(input_dict)

predictionList = local_model.predict(input_dict)
print(predictionList)

prob = tf.nn.softmax(predictionList[0])
print(f"{(100 * prob)} % at {np.argmax(prob, axis=0)} as Severity") 


{'sla': ['24x7 6Hrs Resolution Time'], 'product_type': ['Storage'], 'brand': ['NetApp'], 'service_type': ['Incident'], 'incident_type': ['General Incident'], 'open_to_close_hour': [1268.9333333333334], 'response_to_resolved_hour': [1268.8]}
convert pain data to serdor as input to predict
{'sla': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([[b'24x7 6Hrs Resolution Time']], dtype=object)>, 'product_type': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([[b'Storage']], dtype=object)>, 'brand': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([[b'NetApp']], dtype=object)>, 'service_type': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([[b'Incident']], dtype=object)>, 'incident_type': <tf.Tensor: shape=(1, 1), dtype=string, numpy=array([[b'General Incident']], dtype=object)>, 'open_to_close_hour': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1268.9333]], dtype=float32)>, 'response_to_resolved_hour': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1268.8]

# Get prepared the serving function signature input/output for explanation

In [5]:
input_name = local_model.input_names
print("Model input name:", input_name)
output_name = local_model.output_names
print("Model output name:", output_name)

Model input name: ['open_to_close_hour', 'response_to_resolved_hour', 'sla', 'product_type', 'brand', 'service_type', 'incident_type']
Model output name: ['dense_1']


In [6]:
loaded = tf.saved_model.load(model_dir)

In [7]:
listServingInput= list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys())
print(listServingInput)
serving_input = listServingInput[0]
print(serving_input)

print("Serving function input:", serving_input)

['brand', 'incident_type', 'response_to_resolved_hour', 'open_to_close_hour', 'service_type', 'product_type', 'sla']
brand
Serving function input: brand


In [8]:
listServingOutput=list(loaded.signatures["serving_default"].structured_outputs.keys())
print(listServingOutput)
serving_output =listServingOutput[0]
print("Serving function output:", serving_output)

['dense_1']
Serving function output: dense_1


# Explanation Specification
To get explanations when doing a prediction, you must enable the explanation capability and set corresponding settings when you upload your custom model to an Vertex Model resource. These settings are referred to as the explanation metadata, which consists of:

* parameters: This is the specification for the explainability algorithm to use for explanations on your model. You can choose between:
  * Shapley - Note, not recommended for image data -- can be very long running
  * XRAI
  * Integrated Gradients
* metadata: This is the specification for how the algoithm is applied on your custom model.


In [9]:
XAI = "shapley"  # [ shapley, ig, xrai ]
path_count=35 # 70% of 50 [1-50]
if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": path_count}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": path_count}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": path_count}}

parameters = aip.explain.ExplanationParameters(PARAMETERS)

In [10]:
from google.cloud.aiplatform.explain.metadata.tf.v2 import saved_model_metadata_builder
builder = saved_model_metadata_builder.SavedModelMetadataBuilder(model_dir)
metadata = builder.get_metadata_protobuf()
print(metadata)

# import explainable_ai_sdk
# from explainable_ai_sdk.metadata.tf.v2 import SavedModelMetadataBuilder

# metadata_and_model_builder = SavedModelMetadataBuilder(model_dir)
# metadata_and_model_builder.save_model_with_metadata(model_with_meta_dir)

# https://cloud.google.com/vertex-ai/docs/explainable-ai/getting-explanations#local-explanations
# error
# # Load the model and adjust the configuration for Explainable AI parameters
# # https://cloud.google.com/vertex-ai/docs/reference/rest/v1/ExplanationSpec#sampledshapleyattribution
# num_paths = 25
# model_artifact_with_metadata = explainable_ai_sdk.load_model_from_local_path(
#     model_with_meta_dir,explainable_ai_sdk.SampledShapleyConfig(num_paths))

# instances = [sample ]
# explanations = model_artifact_with_metadata.explain(instances)
# explanations[0].visualize_attributions()

#AttributeError: module 'explainable_ai_sdk' has no attribute 'SampledShapleyConfig'

inputs {
  key: "brand"
  value {
    input_tensor_name: "brand"
    modality: "categorical"
  }
}
inputs {
  key: "incident_type"
  value {
    input_tensor_name: "incident_type"
    modality: "categorical"
  }
}
inputs {
  key: "open_to_close_hour"
  value {
    input_tensor_name: "open_to_close_hour"
  }
}
inputs {
  key: "product_type"
  value {
    input_tensor_name: "product_type"
    modality: "categorical"
  }
}
inputs {
  key: "response_to_resolved_hour"
  value {
    input_tensor_name: "response_to_resolved_hour"
  }
}
inputs {
  key: "service_type"
  value {
    input_tensor_name: "service_type"
    modality: "categorical"
  }
}
inputs {
  key: "sla"
  value {
    input_tensor_name: "sla"
    modality: "categorical"
  }
}
outputs {
  key: "dense_1"
  value {
    output_tensor_name: "dense_1"
  }
}



# Upload model to model registry

In [11]:
# # https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers
# #https://cloud.google.com/vertex-ai/docs/samples/aiplatform-upload-model-sample
image_uri="asia-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-12:latest"
model = aip.Model.upload(
    display_name="model-incident-tf-explainable",
    artifact_uri=model_dir,
    serving_container_image_uri=image_uri,
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    location=region,
    sync=False,
)

model.wait()

Creating Model
Create Model backing LRO: projects/780371717407/locations/asia-southeast1/models/2354195132556771328/operations/6596274819749117952
Model created. Resource name: projects/780371717407/locations/asia-southeast1/models/2354195132556771328@1
To use this Model in another session:
model = aiplatform.Model('projects/780371717407/locations/asia-southeast1/models/2354195132556771328@1')


# Deploy Model

In [12]:
DEPLOYED_NAME = "endpoint-incident-tf-explainable"
TRAFFIC_SPLIT = {"0": 100}
MIN_NODES = 1
MAX_NODES = 1

MACHINE_TYPE = "n1-standard"
VCP = "2"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCP

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=DEPLOY_COMPUTE,
    accelerator_count=0,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

Creating Endpoint
Create Endpoint backing LRO: projects/780371717407/locations/asia-southeast1/endpoints/2469916532356939776/operations/7544282541310607360
Endpoint created. Resource name: projects/780371717407/locations/asia-southeast1/endpoints/2469916532356939776
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/780371717407/locations/asia-southeast1/endpoints/2469916532356939776')
Deploying model to Endpoint : projects/780371717407/locations/asia-southeast1/endpoints/2469916532356939776
Deploy Endpoint model backing LRO: projects/780371717407/locations/asia-southeast1/endpoints/2469916532356939776/operations/610990914973728768
Endpoint model deployed. Resource name: projects/780371717407/locations/asia-southeast1/endpoints/2469916532356939776


# Load model from endpoint to make prediction

In [100]:
instances=[
 {"sla": ["24x7 6Hrs Resolution Time"], "product_type": ["Storage"], "brand": ["NetApp"], "service_type": ["Incident"], "incident_type": ["General Incident"], "open_to_close_hour": [1268.9333333333334], "response_to_resolved_hour": [1268.8]},
 {"sla": ["24x7 4Hrs Resolution Time"], "product_type": ["Software"], "brand": ["Veeam"], "service_type": ["Incident"], "incident_type": ["Software"], "open_to_close_hour": [16.766666666666666], "response_to_resolved_hour": [16.666666666666668]},
# {"sla": ["24x7 4Hrs Resolution Time"], "product_type": ["Server"], "brand": ["HPE"], "service_type": ["Incident"], "incident_type": ["General Incident"], "open_to_close_hour": [1.9], "response_to_resolved_hour": [1.8166666666666667]} 
]

In [101]:
aiplatform.init(project=project_id, location=region)
endpoint_id="1843916184152440832"
endpoint = aiplatform.Endpoint(endpoint_name=f"projects/{project_id}/locations/{region}/endpoints/{endpoint_id}")
endpoint.name

'1843916184152440832'

In [102]:
response = endpoint.predict(instances=instances)
for prediction_ in response.predictions:
        print(prediction_)


[0.0616235808, 0.631787777, 0.298960447, 0.00762823876]
[0.00353190163, 0.0667497888, 0.922751129, 0.00696708122]


# Get explanations
* https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-online-predictions#interpret_explanation_results
* https://cloud.google.com/vertex-ai/docs/tabular-data/classification-explanations


In [85]:
# response = endpoint.explain(instances)
# response

In [104]:
def explain_model( instanceList):

    response = endpoint.explain(instances=instanceList, parameters={})

    for explanation in response.explanations:
        print("Explanation")
        # Feature attributions.
        attributions = explanation.attributions
        for attribution in attributions:
            print("Attribution")
            print(" baseline_output_value:", attribution.baseline_output_value)
            print(" instance_output_value:", attribution.instance_output_value)
            print(" approximation_error:", attribution.approximation_error)
            print(" feature list:")
            sum_feat=0;
            for name in input_name:
              feat_value= attribution.feature_attributions[name]
              sum_feat=sum_feat+feat_value[0]
              print(f"  {name} :{feat_value}")
            print(f"  The sum of all of the feature importance values(instance-baseline) = {sum_feat}")
            
            # print(" output_display_name:", attribution.output_display_name)
            # print("  output_name:", attribution.output_name)
            output_index = attribution.output_index
            for output_index in output_index:
                print(" output_index:", output_index)
            print("================================================================")
    

    for prediction in response.predictions:
        print(prediction)
explain_model(instances)

Explanation
Attribution
 baseline_output_value: 0.28924238681793213
 instance_output_value: 0.6317877173423767
 approximation_error: 0.0007842117239539321
 feature list:
  open_to_close_hour :[0.1434287837147713]
  response_to_resolved_hour :[-0.1710035300254822]
  sla :[0.06976893305778503]
  product_type :[0.07127421796321869]
  brand :[0.2291121137142181]
  service_type :[-0.04315219312906265]
  incident_type :[0.04311700612306595]
  The sum of all of the feature importance values(instance-baseline) = 0.34254533141851423
 output_index: 1
Explanation
Attribution
 baseline_output_value: 0.45522618293762207
 instance_output_value: 0.9227511882781982
 approximation_error: 0.0006550258838518107
 feature list:
  open_to_close_hour :[-0.001835397481918335]
  response_to_resolved_hour :[0.0003963494300842285]
  sla :[0.05224275827407837]
  product_type :[0.004122594594955445]
  brand :[0.1235325539112091]
  service_type :[0.1742244112491608]
  incident_type :[0.1148417353630066]
  The sum o

In [99]:
# response = endpoint.explain(instances=instances, parameters={})
# for explanation in response.explanations:
#  print(explanation.attributions)
#  print("==========================================================")

# Examine feature attributions

In [87]:

import numpy as np


def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training
    #  set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1
     

baseline: 0.28924238681793213
Sanity Check 1: Passed
1  out of  1  sanity checks passed.
baseline: 0.45522618293762207
Sanity Check 1: Passed
1  out of  1  sanity checks passed.
baseline: 0.45522618293762207
Sanity Check 1: Passed
1  out of  1  sanity checks passed.


# Clean 

In [105]:
endpoint.undeploy_all()
delete_bucket = False

endpoint.delete()
# model.delete()

Undeploying Endpoint model: projects/780371717407/locations/asia-southeast1/endpoints/1843916184152440832
Undeploy Endpoint model backing LRO: projects/780371717407/locations/asia-southeast1/endpoints/1843916184152440832/operations/6508454627015393280
Endpoint model undeployed. Resource name: projects/780371717407/locations/asia-southeast1/endpoints/1843916184152440832
Deleting Endpoint : projects/780371717407/locations/asia-southeast1/endpoints/1843916184152440832
Delete Endpoint  backing LRO: projects/780371717407/locations/asia-southeast1/operations/7895563312245506048
Endpoint deleted. . Resource name: projects/780371717407/locations/asia-southeast1/endpoints/1843916184152440832


# Copy Model From Local To GCS

In [192]:
# # #https://codelabs.developers.google.com/codelabs/fraud-detection-ai-explanations?hl=en#6
# # press_y3=input(f"Press y=True to save model to Google Cloud storage : ") 
# # if press_y3.lower()=='y':
# MODEL_BUCKET = 'gs://tf1-incident-smart-ml-yip'

# # # # !gsutil mb -l $REGION $MODEL_BUCKET
# # !gsutil -m cp -r ./$model_dir/* $MODEL_BUCKET/demo_model
# !gsutil -m cp -r ./$model_dir/* $MODEL_BUCKET/model
# #!gsutil -m cp -r ./$explain_meta_model_dir/* $MODEL_BUCKET/demo_model_explain_meta
# # else:
# #  quite()


Copying file://.\model\fingerprint.pb [Content-Type=application/octet-stream]...
Copying file://.\model\saved_model.pb [Content-Type=application/octet-stream]...
/ [0/5 files][    0.0 B/523.0 KiB]   0% Done                                    
Copying file://.\model\keras_metadata.pb [Content-Type=application/octet-stream]...
/ [0/5 files][    0.0 B/523.0 KiB]   0% Done                                    
Copying file://.\model\variables\variables.index [Content-Type=application/octet-stream]...
/ [0/5 files][    0.0 B/523.0 KiB]   0% Done                                    
Copying file://.\model\variables\variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
/ [0/5 files][    0.0 B/523.0 KiB]   0% Done                                    
/ [0/5 files][    0.0 B/523.0 KiB]   0% Done                                    
/ [1/5 files][ 38.9 KiB/523.0 KiB]   7% Done                                    
/ [2/5 files][523.0 KiB/523.0 KiB]  99% Done                          